In [5]:
import os
for v in ("OMP_NUM_THREADS","OPENBLAS_NUM_THREADS","MKL_NUM_THREADS",
          "VECLIB_MAXIMUM_THREADS","NUMEXPR_MAX_THREADS"):
    os.environ[v] = "1"
os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"

from threadpoolctl import threadpool_info
print(threadpool_info())  # every backend should list num_threads=1


[{'user_api': 'blas', 'internal_api': 'openblas', 'num_threads': 1, 'prefix': 'libopenblas', 'filepath': '/opt/anaconda3/envs/grn/lib/libopenblas.0.dylib', 'version': '0.3.29', 'threading_layer': 'openmp', 'architecture': 'VORTEX'}, {'user_api': 'openmp', 'internal_api': 'openmp', 'num_threads': 1, 'prefix': 'libomp', 'filepath': '/opt/anaconda3/envs/grn/lib/libomp.dylib', 'version': None}, {'user_api': 'scipy', 'internal_api': 'scipy_mmio', 'num_threads': 0, 'prefix': '_fmm_core', 'filepath': '/opt/anaconda3/envs/grn/lib/python3.10/site-packages/scipy/io/_fast_matrix_market/_fmm_core.cpython-310-darwin.so', 'version': <function _fmm_version at 0x129d22a70>}]


In [ ]:

import scanpy as sc
import numpy as np
import warnings
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import joblib, json, pandas as pd, itertools, warnings, tqdm, math, pickle
import scipy.io
import scipy.sparse
import anndata
from pathlib import Path
from scipy import sparse


warnings.filterwarnings("ignore", category=FutureWarning)

DATA_ROOT = Path(
    "/Users/yzc/Desktop/Spring2025/CSCI1470/Final Project/RL-in-GRNs/data/GSE132188_RAW"
)
RUN1_DIR  = DATA_ROOT / "mm10"          # use replicate-1 for PoC
OUT_DIR   = Path("./data_prepared")      # everything save will live here
OUT_DIR.mkdir(exist_ok=True)

**1. Load 10× matrix → AnnData**

In [ ]:
# 1_load_10x.py
adata = sc.read_10x_mtx(RUN1_DIR, gex_only=False)  # sparse AnnData
adata.var_names_make_unique()
print(adata)


AnnData object with n_obs × n_vars = 11183 × 27998
    var: 'gene_ids'


**2. QC & normalization**

In [9]:
# 2_qc_normalise.py

import scanpy as sc

# 2.1 Identify mitochondrial genes (human: "MT-"; mouse: "mt-")
adata.var['mt'] = adata.var_names.str.upper().str.startswith('MT-')

# 2.2 Compute QC metrics, including percent mitochondrial counts
sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=['mt'],       # uses our 'mt' column to compute pct_counts_mt
    inplace=True
)

# 2.3 Filter cells
#    - keep cells with > 500 detected genes
#    - drop cells with > 10% mitochondrial reads
adata = adata[adata.obs.n_genes_by_counts > 500, :]
adata = adata[adata.obs.pct_counts_mt < 10, :]

# 2.4 Library‐size normalisation and log1p transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

print(adata)


/var/folders/wz/_2x6lz2x437d29bql459532c0000gn/T/ipykernel_2264/3389961315.py:6: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['mt'] = adata.var_names.str.upper().str.startswith('MT-')
/opt/anaconda3/envs/grn/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


AnnData object with n_obs × n_vars = 11122 × 27998
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mt'
    uns: 'log1p'


**3. Choose gene panel (≤ 100 genes)**

In [10]:
# 3_select_genes.py
TARGET_N_GENES = 100

sc.pp.highly_variable_genes(
    adata, n_top_genes=TARGET_N_GENES, flavor="seurat_v3", subset=True
)
selected_genes = adata.var_names.tolist()       # keep for later
print("Selected", len(selected_genes), "genes")


/opt/anaconda3/envs/grn/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:61: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Selected 100 genes


**4. Binarize expression**

In [12]:
# 4_binarise.py
from scipy import sparse

# 4.1  Convert to a dense NumPy array
if sparse.issparse(adata.X):
    X_dense = adata.X.toarray()
else:
    X_dense = adata.X

# 4.2  Define median‐split threshold function
def median_split(vec: np.ndarray) -> float:
    return np.median(vec)

# 4.3  Compute per‐gene thresholds
thresholds = np.apply_along_axis(median_split, 0, X_dense)

# 4.4  Binarise: cells × genes → 0/1
bin_X = (X_dense > thresholds).astype(np.uint8)
print("Binary matrix:", bin_X.shape, "dtype:", bin_X.dtype)

# 4.5  Store in AnnData for convenience
adata.layers["bin"] = bin_X


Binary matrix: (11122, 100) dtype: uint8


**5. Infer Boolean rules → logic_func_data**

In [13]:
# 5_infer_boolean_rules.py
from scipy.stats import pearsonr

def find_topk_inputs(bin_mat, k=2):
    """Return list[list[int]]; each sub-list are indices of top-k correlated genes."""
    n_genes = bin_mat.shape[1]
    inputs = []
    for g in range(n_genes):
        # compute |corr| with every other gene
        corrs = [
            (abs(pearsonr(bin_mat[:, g], bin_mat[:, j])[0]), j)
            if g != j else (0.0, j)
            for j in range(n_genes)
        ]
        topk = [j for _, j in sorted(corrs, reverse=True)[:k]]
        inputs.append(topk)
    return inputs

top_inputs = find_topk_inputs(bin_X, k=2)

def make_logic(expr_inputs, gene_idx):
    # simple rule: gene g activates if ALL its two regulators are active
    in1, in2 = expr_inputs
    g1, g2 = selected_genes[in1], selected_genes[in2]
    expr = f"{g1} and {g2}"
    return [(expr, 0.8), ("False", 0.2)]   # 80 % ON when both inputs on

logic_func_data = {
    selected_genes[g]: make_logic(top_inputs[g], g)
    for g in range(len(selected_genes))
}
print("Built Boolean rules for", len(logic_func_data), "genes")

# save
with open(OUT_DIR / "logic_func_data.pkl", "wb") as fh:
    pickle.dump(logic_func_data, fh)
pd.Series(selected_genes).to_csv(OUT_DIR / "gene_names.txt", index=False, header=False)


Built Boolean rules for 100 genes


**6. Train / test split for initial states**

In [14]:
# 6_split_states.py
train_states, test_states = train_test_split(
    bin_X, test_size=0.2, random_state=42, shuffle=True
)
np.save(OUT_DIR / "train_states.npy", train_states)
np.save(OUT_DIR / "test_states.npy",  test_states)
print("Train  ", train_states.shape)
print("Test   ", test_states.shape)


Train   (8897, 100)
Test    (2225, 100)


**7. Summarise artifacts**

In [15]:
# 7_summary.py
summary = {
    "n_cells": int(adata.n_obs),
    "n_genes_total": int(adata.n_vars),
    "selected_genes": len(selected_genes),
    "train_cells": len(train_states),
    "test_cells": len(test_states),
}
print(json.dumps(summary, indent=2))
with open(OUT_DIR / "summary.json", "w") as fh:
    json.dump(summary, fh, indent=2)


{
  "n_cells": 11122,
  "n_genes_total": 100,
  "selected_genes": 100,
  "train_cells": 8897,
  "test_cells": 2225
}


In [1]:
# fix_logic_names.py
import pickle, json, numpy as np, re
from pathlib import Path

DATA_DIR   = Path("./data_prepared")
OLD_LOGIC  = DATA_DIR / "logic_func_data.pkl"
OLD_NAMES  = DATA_DIR / "gene_names.txt"
NEW_LOGIC  = DATA_DIR / "logic_func_data_safe.pkl"
NEW_NAMES  = DATA_DIR / "gene_names_safe.txt"

logic = pickle.loads(OLD_LOGIC.read_bytes())
safe   = {}
rename = {}                          # old → new

for old in logic.keys():
    # starts with letter? keep; else prefix with g_
    new = old if re.match(r"[A-Za-z_]", old) else f"g_{old}"
    rename[old] = new

# ─ build new dict with safe keys + update expressions ─
def replace_symbols(expr, renamer):
    # token-wise replace; safe because symbols can only be [A-Z0-9_]
    tokens = re.split(r"(\W)", expr)   # keep delimiters
    return "".join(renamer.get(tok, tok) for tok in tokens)

for old_name, funclist in logic.items():
    new_funclist = []
    for expr, p in funclist:
        new_expr = replace_symbols(expr, rename)
        new_funclist.append((new_expr, p))
    safe[rename[old_name]] = new_funclist

# save artefacts
pickle.dump(safe, NEW_LOGIC.open("wb"))
Path(NEW_NAMES).write_text("\n".join(safe.keys()))
print("Saved", NEW_LOGIC, "and", NEW_NAMES)


Saved data_prepared/logic_func_data_safe.pkl and data_prepared/gene_names_safe.txt
